# Manage Quota

Note: 
- Geo Engine has to have "pro" feature enabled
- quota_check has to be enabled in Settings

In [25]:
import geoengine as ge

from datetime import datetime

# Select a user

In [26]:
email = "foo@example.com"
password = "secret123"

# register the user, if it doesn't exist yet
import requests as re
re.post("http://localhost:3030/api/user", json={"email": email, "password": password, "realName": "Foo Bar"})

<Response [200]>

## Initialize as user

In [27]:
ge.initialize("http://localhost:3030/api", credentials=(email, password))

user_id = ge.get_session().user_id

## Access own quota

In [28]:
ge.get_quota()

{'available': 0, 'used': 0}

## Try to run a query (fails, because quota is exhausted)

In [29]:
ports = ge.register_workflow({
    "type": "Vector",
    "operator": {
        "type": "OgrSource",
        "params": {
            "data": {
                "type": "internal",
                "datasetId": "a9623a5b-b6c5-404b-bc5a-313ff72e4e75"
            },
            "attributeProjection": None
        }
    }
})

time = datetime.strptime(
    '2014-04-01T12:00:00.000Z', "%Y-%m-%dT%H:%M:%S.%f%z")

try:
    df = ports.get_dataframe(
        ge.QueryRectangle(
            ge.BoundingBox2D(-180.0, -90.0, 180.0, 90.0),
            ge.TimeInterval(time, time),
            ge.SpatialResolution(0.1, 0.1)
            )
    )
except ge.GeoEngineException as e:
    print(e)

{'id': '69cf7aaf-e828-537e-be9b-31933120c931'}
Operator: Operator: CreatingProcessorFailed: QuotaExhausted


## Initialize Geo Engine as Admin

In [30]:
ge.initialize("http://localhost:3030/api", ("admin@localhost", "admin"))

## Access user quota

In [31]:
ge.get_quota(user_id)

{'available': 0, 'used': 0}

## Update user quota

In [32]:
ge.update_quota(user_id, 1000)

## Verify quota update worked

In [33]:
ge.get_quota(user_id)

{'available': 1000, 'used': 0}

# Go back to the regular user

In [34]:
ge.initialize("http://localhost:3030/api", credentials=(email, password))

ge.get_quota()

{'available': 1000, 'used': 0}

# Rerun the workflow, works now

In [35]:

df = ports.get_dataframe(
        ge.QueryRectangle(
            ge.BoundingBox2D(-180.0, -90.0, 180.0, 90.0),
            ge.TimeInterval(time, time),
            ge.SpatialResolution(0.1, 0.1)
            )
    )

df.head()

geometry featurecla                          name  \
0   POINT (-69.92356 12.43750)       Port                 Sint Nicolaas   
1  POINT (-58.95141 -34.15333)       Port                       Campana   
2  POINT (-59.00495 -34.09889)       Port                        Zarate   
3  POINT (-62.10088 -38.89444)       Port  Puerto Belgrano/Bahia Blanca   
4  POINT (-62.30053 -38.78306)       Port    Puerto Galvan/Bahia Blanca   

   natlscale  scalerank                          website start end  
0        5.0          8  www.rocargo.com/SanNicolas.html   NaT NaT  
1        5.0          8      www.consejoportuario.com.ar   NaT NaT  
2        5.0          8      www.consejoportuario.com.ar   NaT NaT  
3        5.0          8                             None   NaT NaT  
4        5.0          8                             None   NaT NaT

## Verify that the used quota was recorded

In [36]:
ge.get_quota()

{'available': 999, 'used': 1}